In [1]:
import sys
from pathlib import Path
from collections import defaultdict

import pandas as pd

sys.path.append(str(Path.cwd().parents[1]))

from train_part import ART_DIR
from source.trackers import CVTracker
from source.general_functions import update_results_dict
from configs.variational_inference import (
    FMAP,
    MODELS,
    CV_NAME, 
    RES_NAME, 
    DATASETS, 
)

def get_res_dir(dir_path, model, data, fmap):
    return Path(
        dir_path / f'{model}/{data}/{fmap}/'
    )

### Table with Metrics:

In [ ]:
res_dict = defaultdict(list)
for data_name in DATASETS:
    for model_name in MODELS:
        res_dir = get_res_dir(ART_DIR, model_name, data_name, FMAP)
        tracker = CVTracker(res_dir, RES_NAME, CV_NAME)
        res_df, _ = tracker.load()
        nll_str = f"{res_df['nll_test'].mean():.2f}±{res_df['nll_test'].std():.2f}"
        time_str = f"{res_df['train_time'].mean():.1f}±{res_df['train_time'].std():.1f}"
        update_results_dict(
            res=res_dict, data=data_name.capitalize(), model=model_name.upper(), 
            nll_test=nll_str, time=time_str,
        )

res_df = pd.DataFrame(res_dict)
res_df.columns = ['Data', 'Model', 'NLL Test', 'Time (s)']
res_df = res_df.pivot(index='Data', columns='Model', values=['NLL Test', 'Time (s)'])

caption = 'Test NLL results and training time of the LA-TTKM model using VI or CV.'
print(
    (
        res_df
        .to_latex( 
            column_format='||l||c|c||c|c||',
            caption=caption,
            label="table:vi_cv",
            multirow=True,
            index=True,
        )
    )
);